In [1]:
dividers = [x for x in range(1, 1056+1) if 1056 % x == 0]

In [2]:
dividers

[1,
 2,
 3,
 4,
 6,
 8,
 11,
 12,
 16,
 22,
 24,
 32,
 33,
 44,
 48,
 66,
 88,
 96,
 132,
 176,
 264,
 352,
 528,
 1056]

In [5]:
!pip install geopy

In [2]:
!pip install geopy[aiohttp]

In [8]:
!pip install geopandas

In [1]:
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import numpy as np
db_v4 ='https://drive.google.com/file/d/19ERs5bmAdxEfgUmTxgfIBhUoT6xPHzZy/view?usp=sharing'
fname1 = db_v4
fname1='https://drive.google.com/uc?id=' + fname1.split('/')[-2]
dataset = pd.read_csv(fname1, low_memory=False)
idx = np.where((dataset["Proposed Use"]=="1 family dwelling") | (dataset["Proposed Use"]=="2 family dwelling") | (dataset["Proposed Use"]=="apartments"))
dataset = dataset.loc[idx]

In [2]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7662 entries, 0 to 9407
Data columns (total 42 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Permit Number                           7662 non-null   object 
 1   Permit Type                             7662 non-null   int64  
 2   Permit Type Definition                  7662 non-null   object 
 3   Permit Creation Date                    7662 non-null   object 
 4   Block                                   7662 non-null   object 
 5   Lot                                     7662 non-null   object 
 6   Street Number                           7662 non-null   int64  
 7   Street Name                             7662 non-null   object 
 8   Street Suffix                           7662 non-null   object 
 9   Description                             6507 non-null   object 
 10  Current Status                          7662 non-null   obje

0
(7662, 42)


In [7]:
import time
# Get geometry from Nominatim
geometry_list = []
print(len(geometry_list))
print(dataset.shape)

print('Get geometry from Nominatim')
start_time = time.time()
with Nominatim(
    user_agent="Building Permits San Francisco",
) as geolocator:
    for row in range(len(dataset.values)):
        address = dataset.iloc[row]["address"] + " SF US"
        try:
            location = geolocator.geocode(address, language="en", geometry="geojson")
            geometry_list.append(location.raw["geojson"])
            print(f"ok: {address}")
        except:
            print(f"except: {address}")
            try :
                address = str(dataset.iloc[row]["Street Number"]) +" "+ dataset.iloc[row]["Street Name"] + " SF US"
                location = geolocator.geocode(address, language="en", geometry="geojson")
                geometry_list.append(location.raw["geojson"])
                print(f"except ok{address}")
            except:
                # If Nominatim doesn't find the information so we fill with nan value
                geometry_list.append(np.nan)
                print(f"except ko{address}")
                
print("--- %s seconds ---" % (time.time() - start_time))



0
(7662, 42)
Get geometry from Nominatim
ok: 3686 17th St SF US
ok: 1254 18th Av SF US
ok: 290 Melrose Av SF US
except: 1059 Union St SF US
except ko1059 Union SF US
ok: 3900 California St SF US
ok: 1084 De Haro St SF US
ok: 1082 De Haro St SF US
ok: 1084 De Haro St SF US
ok: 1570 9th Av SF US
ok: 133 Yukon St SF US
ok: 2929 Geary Blvd SF US
ok: 26 Balboa St SF US
ok: 1616 Funston Av SF US
ok: 530 36th Av SF US
except: 1228 46th Av SF US
except ko1228 46th SF US
ok: 950 Columbus Av SF US
ok: 1123 20th St SF US
ok: 361 Franconia St SF US
ok: 404 Franconia St SF US
ok: 985 Girard St SF US
except: 1 Locust St SF US
except ko1 Locust SF US
ok: 3505 Pacific Av SF US
ok: 321 Clementina St SF US
ok: 401 Grand View Av SF US
ok: 2593 23rd Av SF US
ok: 4537 25th St SF US
ok: 1368 De Haro St SF US
ok: 315 28th Av SF US
ok: 418 2nd Av SF US
ok: 720 Gough St SF US
ok: 1727 Fillmore St SF US
ok: 1727 Fillmore St SF US
ok: 1525 18th St SF US
ok: 322 19th Av SF US
ok: 1924 25th St SF US
ok: 242 Munich

NameError: name 'polygon_list' is not defined

In [11]:
# Create the lat and lon features for [permit_noLoc] dataset
dataset["geometry"] = geometry_list

In [15]:
dataset.drop(columns = "polygon", inplace=True)

In [30]:
dataset.to_csv('Building_Permits_v6.csv',index=False)

In [ ]:
test = dataset.dropna(subset="geometry")
len(test)

In [29]:
async with Nominatim(
    user_agent="Building Permits San Francisco",
    adapter_factory=AioHTTPAdapter,
) as geolocator:
    location = await geolocator.geocode("175 5th Avenue NYC", country_codes="US", geometry="geojson")
    print(location.address.split(","))
    print(location.raw["geojson"])

state_l = []
geometry_l = []
d = {'state': [1, 2], 'geometry': [3, 4]}
df = pd.DataFrame(data=d)

['Flatiron Building', ' 175', ' 5th Avenue', ' Manhattan Community Board 5', ' Manhattan', ' New York County', ' City of New York', ' New York', ' 10010', ' United States']
{'type': 'Polygon', 'coordinates': [[[-73.9898715, 40.7408907], [-73.9898703, 40.7408821], [-73.989866, 40.7408739], [-73.9898591, 40.740867], [-73.9898548, 40.7408641], [-73.9896191, 40.7407635], [-73.9896147, 40.7407617], [-73.9896051, 40.7407597], [-73.9895954, 40.7407566], [-73.9895903, 40.740761], [-73.9895815, 40.7407645], [-73.9895745, 40.7407697], [-73.9895699, 40.7407764], [-73.9895356, 40.7410206], [-73.9895014, 40.7412649], [-73.989507, 40.7412827], [-73.9895144, 40.7412898], [-73.989519, 40.7412928], [-73.9895293, 40.7412974], [-73.9895349, 40.7412989], [-73.9895467, 40.7413004], [-73.9895586, 40.7412995], [-73.9895644, 40.7412983], [-73.9895751, 40.7412942], [-73.9895841, 40.7412883], [-73.9897246, 40.7410978], [-73.989865, 40.7409073], [-73.9898697, 40.7408994], [-73.9898715, 40.7408907]]]}
